In [6]:
import json
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import numpy as np
import inflect
engine = inflect.engine()

In [7]:
det_ingr = json.load(open("../generated/1m_valid_recipes.json"))

In [8]:
def singularize(word):
    ingr = engine.singular_noun(word)
    return word if (not ingr) else ingr

def clean_ing_word(word) : return singularize(word).lower()

def clean_whole_ing(ing) : return " ".join([clean_ing_word(word) for word in ing.split(" ")])

In [9]:
embeddings_train = []
for r in tqdm_notebook(det_ingr[:2]) :
    embeddings_train.append([clean_whole_ing(ing['text']) for e, ing in enumerate(r['ingredients']) if r['valid'][e]])
    
print(embeddings_train)


[['elbow macaroni', 'american cheese', 'celery', 'green pepper', 'pimento', 'mayonnaise', 'vinegar', 'salt', 'dry dill weed'], ['tomato', 'kosher salt', 'red onion', 'green bell pepper', 'red bell pepper', 'yellow bell pepper', 'cucumber', 'olive oil', 'fresh basil']]


Word2Vec training

In [ ]:
start_time = time.time()
model = gensim.models.Word2Vec(embeddings_train, min_count=2)

time_after_creation = time.time()
print("creation of model :", time_after_creation - start_time, "sec.")

model.train(embeddings_train, total_examples=len(embeddings_train), epochs=10)

time_after_training = time.time()
print("model training :", time_after_training - time_after_creation, "sec.")

model.save('food_embeddings.bin')

time_after_saving = time.time()
print("model saving :", time_after_saving - time_after_training, "sec.")

Results investigation

In [ ]:
model.wv.most_similar("mushroom")

We see that the adjectives are creating new ingredients. Given we are only interested in flavours associations, we should ignore them. Indeed, shredded mushrooms and dried mushrooms should have the same flavour. 

In [ ]:
for index in range(0, 20) :
    for e, ing in enumerate(det_ingr[index]['ingredients']) :
        if det_ingr[index]['valid'][e] :
            tags = nltk.pos_tag(nltk.word_tokenize(clean_ing(ing['text'])))
            if 'VBN' == tags[0][1] and tags[1][0] != '-' :
                print(tags)

In [ ]:
model.wv.similarity(w1="lamb", w2="beef")

In [ ]:
 model.most_similar(positive=['rice', 'tomato'], negative=['pasta'], topn=1)